In [10]:
import pandas as pd
import re

# 1. 데이터 불러오기


In [11]:
df = pd.read_csv('kpop_2010_2025_curated.csv')

# 2. 불필요한 컬럼 제거
###    (분석에 필요 없는 ID, isrc, 중복 정보 등 삭제)

In [12]:
drop_cols = ['album_name', "artist_id", "album_id", "track_id", "isrc", "duration_min"]

df = df.drop(columns=drop_cols, errors="ignore")

# 3. 변형곡(Variant) 제거
###    - Japanese Ver., Remix, Live, Acoustic, Cover, Edit 등
###    - 원곡만 남기고 중복/왜곡을 줄이기 위함

In [13]:
drop_keywords = [
    "japanese ver", "japanese version", "jp ver",
    "remix", "mix", "edit", "rework", "remake",
    "live", "acoustic", "instrumental", "inst",
    "cover", "ver."
]

In [14]:
pattern = re.compile("|".join(drop_keywords), flags=re.IGNORECASE)
mask = df["track_name"].str.contains(pattern, na=False)
df_clean = df[~mask].copy()

In [15]:
print(f"원본 행 수: {len(df)}")
print(f"제거된 행 수: {mask.sum()}")
print(f"남은 행 수: {len(df_clean)}")

원본 행 수: 810
제거된 행 수: 282
남은 행 수: 528


# 4. track_name 중복 확인
###    - 같은 track_name이 여러 번 존재하는 경우 집계


In [16]:
name_counts = df_clean["track_name"].value_counts()
dup_names = name_counts[name_counts > 1]
print("중복된 곡 수:", len(dup_names))

중복된 곡 수: 114


# 5. 중복 제거
###   - 같은 track_name이 여러 번 있을 경우 popularity가 가장 높은 행만 남기고 나머지는 드랍

In [17]:
df_dedup = df_clean.sort_values("popularity", ascending=False).drop_duplicates(subset=["track_name"], keep="first")

In [18]:
print(f"원본 행 수: {len(df)}")
print(f"중복 제거 후 행 수: {len(df_dedup)}")

원본 행 수: 810
중복 제거 후 행 수: 390


In [26]:
name_counts = df_dedup["track_name"].value_counts()
dup_names = name_counts[name_counts > 1]
print("중복된 곡 수:", len(dup_names))

중복된 곡 수: 0


# 6. 결과 저장

In [20]:
df_dedup.to_csv('kpop_2010_2025_curated_final.csv',
                index=False, encoding="utf-8-sig")
print("최종 전처리 CSV 저장 완료 → Spotify_project/data/kpop_2010_2025_curated_final.csv")

최종 전처리 CSV 저장 완료 → Spotify_project/data/kpop_2010_2025_curated_final.csv


In [22]:
df = pd.read_csv('kpop_2010_2025_curated_final.csv')

In [24]:
df.columns

Index(['artist', 'album_type', 'track_name', 'release_date', 'release_year',
       'popularity', 'duration_ms', 'explicit', 'disc_number', 'track_number',
       'song_age_years', 'staying_index'],
      dtype='object')

In [25]:
df[df['artist'] == 'TXT']

,artist,album_type,track_name,release_date,release_year,popularity,duration_ms,explicit,disc_number,track_number,song_age_years,staying_index
0,TXT,album,Beautiful Strangers,2025-07-21,2025,85,138108,False,1,2,0,85.000000
2,TXT,album,Ghost Girl,2025-07-21,2025,78,186601,False,1,3,0,78.000000
7,TXT,single,Love Language,2025-05-02,2025,76,177514,False,1,1,0,76.000000
11,TXT,single,butterflies (feat. TAEHYUN & Kim Chaewon of LE...,2025-07-10,2025,74,164466,False,1,1,0,74.000000
15,TXT,album,Take My Half,2025-07-21,2025,74,176076,False,1,6,0,74.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
349,TXT,album,Hitori no Yoru,2023-07-05,2023,18,218859,False,1,9,2,6.000000
353,TXT,album,Ring,2023-07-05,2023,17,204111,False,1,6,2,5.666667
379,TXT,album,Intro : FLOATING,2023-07-05,2023,8,111935,False,1,1,2,2.666667
383,TXT,album,Outro : FALLING,2023-07-05,2023,7,129362,False,1,12,2,2.333333
